In [ ]:
!pip install ultraytics
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok config add-authtoken 2WqPAb6RElk7pdtvp34ihETJmEo_2q6hniy8qmDmFkHjjNSsB

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM 
import os
import flask
import torch
from PIL import Image

device = "cuda:0" if torch.cuda.is_available() else "cpu"

florence_model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, trust_remote_code=True).to(device)
florence_processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

MESSAGES = []

def get_conn_id():
    import random
    return "".join(random.choices("qwertyuiopasdfghjklzxcvbnm1234567890QWERTYUIOPASDFGHJKLZXCVBNM", k =20))

app = flask.Flask("AXC")

@app.route("/flAccess", methods=["POST"])
def flAccess():
    con_id = get_conn_id()
    prompt = dict(flask.request.form)["prompt"]
    file = flask.files["files"]
    file.save(f"input{con_id}.png")
    
    inputs = florence_processor(text=prompt, images=Image.open(f"input{con_id}.png"), return_tensors="pt").to(device, torch.float16 if torch.cuda.is_available() else torch.float32)
    generated_ids = florence_model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3
    )

    generated_text = florence_processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = florence_processor.post_process_generation(generated_text, task=prompt, image_size=(image.width, image.height))

    try:
        os.remove(f"input{con_id}.png")
    except:
        pass

    return parsed_answer

@app.route("/alert")
def alerts():
    if len(MESSAGES) == 0:
        return "", 200
    return MESSAGES[-1], 200

@app.route("/add_alert", methods=['POST'])
def add_alerts():
    MESSAGES.append(dict(flask.request.json)["message"])
    return "",200

@app.route("/clear_alerts")
def clear_alerts():
    MESSAGES = []
    return "", 200
import subprocess
ngrok_proc = subprocess.Popen("ngrok http --url=apparent-stirred-bluegill.ngrok-free.app 5000".split())
app.run(port=5000)

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise y

 * Serving Flask app 'AXC'
 * Debug mode: off
